In [1]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np


sns.set()
%matplotlib inline

In [2]:
import unidecode

def normalizeSentence(sentence):
    
    temp = sentence.split(' ')
    new_sentence = []
    for word in temp:
        new_sentence.append(normalizeWord(word))
    new_sentence = ' '.join(new_sentence)
    
    return new_sentence

def normalizeWord(word):
    
    # lower case word
    word = word.lower()
    
    # remove accents
    word = unidecode.unidecode(word)
    
    return word

def joinSeparetedWord(word):
    
    # removing aspas
    word = word.split('\'')
    word = ''.join(word)
    
    # separete words from word
    word = word.split()
    
    # join separeted words by '-'
    word = '-'.join(word)
    
    return word

In [3]:
def selectByWord(word, sentences):
    # select subset of sentences that contains word
    # return indexs of sentences
    
    # normalize word
    normalized_word = normalizeWord(word)
    
    indexs = []
    
    for i, sentence in enumerate(sentences):
        # normalize sentence
        normalized_sentence = normalizeSentence(sentence)
        
        if normalized_word in normalized_sentence:
            #print(i, normalized_sentence)
            indexs.append(i)
    return indexs

In [4]:
def selectDfRowsByWord(data, word, column):
    '''
    Select specific rows that contains especific word in the column
    '''
    
    sentences = np.array(data[column].unique())

    # find sentences
    idxs = selectByWord(word, sentences)
    obj_sentences = sentences[idxs]

    servicos = pd.DataFrame()
    for sentence in obj_sentences:

        # select rows of exemplo
        temp = data[data[column] == sentence]

        # concatenate
        servicos = pd.concat([servicos, temp])
    try:
        temp = len(servicos[column].unique())
    except:
        print(word + ': Not found')
    
    return servicos.copy()

In [85]:
class Municipio:
    '''
    
    Município
    
    '''
    
    def __init__(self, name, years = (2018, 2018)):
        
        self.keep_columns_ = ['ano_exercicio', 'mes_ref_extenso', 'tp_despesa', 'vl_despesa',
                              'ds_funcao_governo', 'ds_subfuncao_governo', 'ds_programa', 'ds_acao']
        self.numeric_columns_ = ['ano_exercicio', 'vl_despesa']
        
        self.years_ = np.arange(years[0], years[1]+1)
        self.name_ = name
        self.df_ = pd.DataFrame()
        self.population_ = None
        print('Reading data...')
        self.readData()
        print('Preparing data...')
        self.prepareData()
        print('\nBase for {} is ready!!'.format(self.name_))
        
    def readData(self):
        
        print('> reading population...')
        # reading population
        
        municipios_base = pd.read_csv('datasets/municipios.csv')
        municipios = municipios_base['Município'].values
        for i, municipio in enumerate(municipios):
            if normalizeWord(self.name_) == normalizeWord(municipio):
                self.population_ = municipios_base['Estimativa 2019'].values[i]
        if self.population_ is None:
            print('Erro: population not found')
            return
        
        print('> reading despesas...')
        # reading despesas
        
        for year in self.years_:
            print('>> year: '+str(year))
            try:
                aux = open('datasets/despesas/'+ str(year) +'/despesas-' + self.name_ + '-'+str(year)+'.csv', 'r')
                aux.close()
                del aux
            except:
                downloadDespesas([self.name_], year, unzip=True)
            temp = pd.read_csv('datasets/despesas/'+ str(year) +'/despesas-' + self.name_ + '-'+str(year)+'.csv', sep = ';',  encoding = "ISO-8859-1")
            temp = temp[self.keep_columns_]
            self.df_ = pd.concat([self.df_, temp]).reset_index(drop = True)
        del temp
        
    def prepareData(self):
        
        # string para float
        self.df_['vl_despesa'] = self.df_.vl_despesa.apply(lambda x : float('.'.join(x.split(','))))
        
        # normalizando nome dos meses
        for column in self.df_.columns:
            if column in self.numeric_columns_:
                 continue
            self.df_[column] = self.df_[column].apply(lambda x : normalizeWord(x))
        
        self.df_['vl_despesa_per_capita'] = self.df_['vl_despesa'].apply(lambda x : x/self.population_)
        
        # ajsutando nomes com . no meio
        for column in ['ds_programa', 'ds_acao']:
            self.df_[column] = self.df_[column].apply(lambda x : ''.join(x.split('.')))
        

In [75]:
from zipfile import ZipFile
import urllib.request
import progressbar
import os

def downloadDespesas(names, year, unzip = False):
        
    n = len(names)
    erros = []
    
    # widgets for progressbar
    widgets=[
        ' [', progressbar.Timer(), '] ',
        progressbar.Bar(),
        ' (', progressbar.Counter(), '/'+str(n)+') ',
    ]

    for i in range(n):
        name = names[i]

        # uniformalizing word for match
        name = normalizeWord(name)

        # replacing ' ' to '-'
        name = joinSeparetedWord(name)

        # request download
        url = 'https://transparencia.tce.sp.gov.br/sites/default/files/csv/despesas-' + name + '-' + str(year) + '.zip'
        try:
            zipName = 'downloads/' + name + '-' + str(year) + '.zip'
            urllib.request.urlretrieve(url, zipName)
            if unzip:
                with ZipFile(zipName, 'r') as zipObj:
                    zipObj.extractall('datasets/despesas/' + str(year))
                try:
                    os.rename('datasets/despesas/' + str(year) + '/despesas-' + name + '-' + str(year) + '.zip.csv', 'datasets/despesas/' + str(year) + '/despesas-' + name + '-' + str(year) + '.csv')
                except:
                    continue
        except:
            erros.append(name)
        
    if len(erros) == 0:
        print('All files have been downloaded!')
    else:
        print('Files not found:')
        print(erros)
    

In [76]:
municipios = pd.read_csv('datasets/municipios_no_sp.csv').set_index('Município')

In [81]:
municipios_nomes = ['miracatu']

In [78]:
sample = municipios.loc[municipios_nomes]

In [79]:
sample

,Posição,Estimativa 2019
Município,,
Miracatu,262,19779


In [72]:
downloadDespesas(sample.index, unzip = True, year = 2016)

All files have been downloaded!


In [86]:
miracatu = Municipio('miracatu', years = (2014, 2018))

Reading data...
> reading population...
> reading despesas...
>> year: 2014
All files have been downloaded!
>> year: 2015
All files have been downloaded!
>> year: 2016
>> year: 2017
All files have been downloaded!
>> year: 2018
All files have been downloaded!
Preparing data...

Base for miracatu is ready!!
